In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

['train', 'labels.csv', 'sample_submission.csv', 'test']


In [2]:

import keras
from keras.applications.vgg19 import VGG19
from keras.models import Model
from keras.layers import Dense, Dropout, Flatten
import tensorflow as tf
import os
from tqdm import tqdm
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import cv2
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

Using TensorFlow backend.


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
from fastai import *
from fastai.vision import *
from fastai.metrics import error_rate

In [5]:
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

labels.csv
sample_submission.csv
test
train



In [6]:
PATH ="../input"

In [7]:
!ls {PATH}

labels.csv  sample_submission.csv  test  train


In [8]:
df_train = pd.read_csv('../input/labels.csv')
df_test = pd.read_csv('../input/sample_submission.csv')

In [9]:
df_train.head(10)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
7,002a283a315af96eaea0e28e7163b21b,borzoi
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound


In [10]:
df_train.shape

(10222, 2)

In [11]:
df_train.breed.value_counts().head(5)

scottish_deerhound      126
maltese_dog             117
afghan_hound            116
entlebucher             115
bernese_mountain_dog    114
Name: breed, dtype: int64

In [12]:
df_train.columns

Index(['id', 'breed'], dtype='object')

In [13]:
df_train.describe()

,id,breed
count,10222,10222
unique,10222,120
top,2958f3aa05c668495b874ea995465060,scottish_deerhound
freq,1,126


In [14]:
df_test.shape

(10357, 121)

In [15]:
df_test.columns

Index(['id', 'affenpinscher', 'afghan_hound', 'african_hunting_dog',
       'airedale', 'american_staffordshire_terrier', 'appenzeller',
       'australian_terrier', 'basenji', 'basset',
       ...
       'toy_poodle', 'toy_terrier', 'vizsla', 'walker_hound', 'weimaraner',
       'welsh_springer_spaniel', 'west_highland_white_terrier', 'whippet',
       'wire-haired_fox_terrier', 'yorkshire_terrier'],
      dtype='object', length=121)

In [16]:
df_test.id.value_counts().head(3)

d52eb0b5629fde69f9fa6d5e8981187b    1
49e4381d88ea41b16cd08e03d706d49f    1
f85dd6bab03d372d771dd7815e732e9d    1
Name: id, dtype: int64

In [17]:
df_test.describe()

,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,english_foxhound,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
count,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,...,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04,1.035700e+04
mean,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,...,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03,8.333333e-03
std,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,...,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.427746e-15,1.4

In [18]:
targets_series = pd.Series(df_train['breed'])
one_hot = pd.get_dummies(targets_series, sparse = True)

In [19]:
one_hot_labels = np.asarray(one_hot)

In [20]:
im_size = 90

In [21]:
x_train = []
y_train = []
x_test = []

In [22]:
i = 0 
for f, breed in tqdm(df_train.values):
    img = cv2.imread('../input/train/{}.jpg'.format(f))
    label = one_hot_labels[i]
    x_train.append(cv2.resize(img, (im_size, im_size)))
    y_train.append(label)
    i += 1

100%|██████████| 10222/10222 [00:40<00:00, 254.11it/s]


In [23]:
for f in tqdm(df_test['id'].values):
    img = cv2.imread('../input/test/{}.jpg'.format(f))
    x_test.append(cv2.resize(img, (im_size, im_size)))

100%|██████████| 10357/10357 [00:41<00:00, 247.65it/s]


In [24]:
y_train_raw = np.array(y_train, np.uint8)
x_train_raw = np.array(x_train, np.float32) / 255.
x_test  = np.array(x_test, np.float32) / 255.

In [25]:
print(x_train_raw.shape)
print(y_train_raw.shape)
print(x_test.shape)

(10222, 90, 90, 3)
(10222, 120)
(10357, 90, 90, 3)


In [26]:
num_class = y_train_raw.shape[1]

In [27]:
X_train, X_valid, Y_train, Y_valid = train_test_split(x_train_raw, y_train_raw, test_size=0.3, random_state=1)